In [1]:
!pip install -q transformers datasets evaluate comet_ml tqdm accelerate scikit-learn 


In [2]:
import sys
print(sys.executable)


C:\Users\Educational Purpose\AppData\Local\Programs\Python\Python313\python.exe


In [3]:
from comet_ml import Experiment
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, pipeline
)
from datasets import Dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
import evaluate
import torch
import threading, time

C:\Users\Educational Purpose\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def keep_awake():
    while True:
        time.sleep(60)
        print(" CPU still active...")

threading.Thread(target=keep_awake, daemon=True).start()

 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still active...
 CPU still ac

In [5]:

import os
os.getcwd()


'C:\\Users\\Educational Purpose\\Downloads'

In [6]:
train_df = pd.read_csv(r"C:\Users\Educational Purpose\Downloads\posts_train.csv")
train_df.head()

,ID,post,class_name,class_id,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,691324c4-5c30-44e0-b9e4-45b4f0715e21,i was making questions for my students and i r...,none,5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,d4295391-9ca5-4398-b7c8-687e4a984ef1,i've recently requested testing accommodations...,adhd,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58937fa5-3c2c-426b-8255-5a140fbab675,**cambodia** * koh rong: amazing beaches and a...,none,5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7daf364c-3b33-4cbe-be37-a214edf9a73e,synesthesia. what is synesthesia? according to...,none,5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22518271-4bb4-4caf-b683-7305da519288,i’m phil baran and i teach organic chemistry a...,none,5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
train_df = train_df.dropna(subset=["post", "class_name"]).reset_index(drop=True)

print(train_df.head())

                                     ID  \
0  691324c4-5c30-44e0-b9e4-45b4f0715e21   
1  d4295391-9ca5-4398-b7c8-687e4a984ef1   
2  58937fa5-3c2c-426b-8255-5a140fbab675   
3  7daf364c-3b33-4cbe-be37-a214edf9a73e   
4  22518271-4bb4-4caf-b683-7305da519288   

                                                post class_name class_id  \
0  i was making questions for my students and i r...       none        5   
1  i've recently requested testing accommodations...       adhd        0   
2  **cambodia** * koh rong: amazing beaches and a...       none        5   
3  synesthesia. what is synesthesia? according to...       none        5   
4  i’m phil baran and i teach organic chemistry a...       none        5   

  Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  ...  \
0        NaN        NaN        NaN        NaN        NaN        NaN  ...   
1        NaN        NaN        NaN        NaN        NaN        NaN  ...   
2        NaN        NaN        NaN        NaN        NaN

In [9]:
train_df = pd.read_csv(r"C:\Users\Educational Purpose\Downloads\posts_train.csv")

In [10]:
train_df = train_df.dropna(subset=["post", "class_name"]).iloc[9150:13729].reset_index(drop=True)
print(train_df.head())
print(train_df.tail())

                                     ID  \
0  71fa22c0-6ffc-4844-9f7a-492d8c1e5eb8   
1  f020837c-4aa7-477d-a133-e2623182a977   
2  24825e3a-9af7-4b43-9bd1-503fd8df9f6f   
3  da944fef-3d5a-4644-bc3d-0c59530410d9   
4  8e370f3e-d502-4c58-83f0-0d4e4fd3da4c   

                                                post class_name class_id  \
0  when i first properly looked at the symptoms a...    bipolar        2   
1  you have all these worries and they keep you a...    anxiety        1   
2  yep,you heard me right. indian is exempted fro...       none        5   
3  today i turn 30. i’m buying a house with my hu...    bipolar        2   
4  she told me to gtfo and never come back yester...    bipolar        2   

  Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  ...  \
0        NaN        NaN        NaN        NaN        NaN        NaN  ...   
1        NaN        NaN        NaN        NaN        NaN        NaN  ...   
2        NaN        NaN        NaN        NaN        NaN

In [14]:
paraphraser = pipeline(
    "text2text-generation",
    model="humarin/chatgpt_paraphraser_on_T5_base",
    device=-1
)

def paraphrase_text(text):
    try:
        result = paraphraser(
            f"paraphrase: {text}",
            max_new_tokens=64,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            num_return_sequences=1
        )
        return result[0]['generated_text']
    except Exception:
        return text




Device set to use cpu


In [15]:
tqdm.pandas()
train_df["post_paraphrased"] = train_df["post"].progress_apply(paraphrase_text)


100%|████████████████████████████████████████████████████████████████████████████| 4577/4577 [9:06:47<00:00,  7.17s/it]


In [ ]:

output_dir = r"C:\Users\Educational Purpose\Downloads"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "posts_train_paraphrased_part3.csv")
train_df.to_csv(output_path, index=False)
print(f" Saved paraphrased part 3 to: {output_path}")

 Saved paraphrased part 3 to: C:\Users\Educational Purpose\Downloads\posts_train_paraphrased_part3.csv


In [19]:
import pandas as pd

In [21]:
df1 = pd.read_csv(r"C:\Users\Educational Purpose\Downloads\posts_train_paraphrased_part1.csv")
df2 = pd.read_csv(r"C:\Users\Educational Purpose\Downloads\posts_train_paraphrased_part2.csv")
df3 = pd.read_csv(r"C:\Users\Educational Purpose\Downloads\posts_train_paraphrased_part3.csv")



In [24]:
print (df1.head())
print (df2.head())
print (df3.head())

                                     ID  \
0  691324c4-5c30-44e0-b9e4-45b4f0715e21   
1  d4295391-9ca5-4398-b7c8-687e4a984ef1   
2  58937fa5-3c2c-426b-8255-5a140fbab675   
3  7daf364c-3b33-4cbe-be37-a214edf9a73e   
4  22518271-4bb4-4caf-b683-7305da519288   

                                                post class_name  class_id  \
0  i was making questions for my students and i r...       none         5   
1  i've recently requested testing accommodations...       adhd         0   
2  **cambodia** * koh rong: amazing beaches and a...       none         5   
3  synesthesia. what is synesthesia? according to...       none         5   
4  i’m phil baran and i teach organic chemistry a...       none         5   

                                    post_paraphrased  
0  As I worked on my questions for my students, I...  
1  I recently requested testing accommodations at...  
2  Despite its remote location, it is still one o...  
3  Can you explain what synesthesia is? It's a co...  
4  

In [25]:
merged_df = pd.concat([df1, df2, df3], ignore_index=True)

In [28]:
merged_df.to_csv(r"C:\Users\Educational Purpose\Downloadsposts_train_paraphrased_full.csv", index=False)